In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
N = torch.zeros((27,27))

In [56]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []

for w in words[:1]:
  context = [0] * block_size
  for ch in (w + '.'):
    X.append(context)
    ix = stoi[ch]
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]
# for w in words:
#   #print(w)
#   context = [0] * block_size # duplicate [0] to [0,0,0]
#   for ch in w + '.':
#     ix = stoi[ch]
#     X.append(context)
#     Y.append(ix)
#     print(''.join(itos[i] for i in context), '--->', itos[ix])
#     context = context[1:] + [ix] # crop and append
#     # context will always fall behind one bit from true label
  
X = torch.tensor(X)
Y = torch.tensor(Y)

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [11]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])

In [12]:
Y

tensor([ 5, 13, 13,  1,  0])

In [13]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([5, 3]), torch.int64, torch.Size([5]), torch.int64)

In [22]:
# A layer of two neurons
C = torch.randn((27, 2))
C

tensor([[-0.8149, -1.4727],
        [-1.6462, -1.4466],
        [ 0.0954,  0.2717],
        [ 0.4230, -0.4369],
        [-1.1684,  2.0900],
        [ 0.6220,  1.0828],
        [-0.1982, -0.0445],
        [ 1.3953,  1.0711],
        [-1.1981, -1.1842],
        [ 0.3029,  1.2056],
        [ 0.8761, -1.2525],
        [ 0.0366,  1.8045],
        [-0.7391,  0.0728],
        [-1.1231,  0.1798],
        [-1.4057,  0.8786],
        [ 1.2651,  1.3113],
        [ 0.5063,  1.7261],
        [-1.1865,  0.8545],
        [ 0.6775, -0.4491],
        [-1.2347, -0.5361],
        [ 0.2071,  0.8200],
        [ 0.7417,  0.4729],
        [ 0.6101, -0.8640],
        [-1.7173, -1.2424],
        [ 0.9097,  0.5655],
        [ 1.3417,  1.5396],
        [ 1.6055,  2.4652]])

In [26]:
'''
This operation is performing indexing into C using X as the index tensor. Here's what happens step by step:

    Indexing with X: For each row in X, the values are treated as indices to fetch rows from C. Since X has 3 columns, each row of X will select 3 rows from C. The result is that for every row in X, you get a corresponding set of 3 embedding vectors from C.

    Shape of the Result (emb): Given that each row in X leads to selecting 3 embedding vectors from C, and each embedding vector has a dimensionality of 2 (since C has shape [27, 2]), the resulting tensor emb will have a shape that reflects the combination of these dimensions. Specifically, the shape of emb will be [228146, 3, 2]. This means there are 228146 groups of embeddings, each group containing 3 embeddings, and each embedding having 2 dimensions.
'''
emb = C[X] # ???? Why ?
emb.shape
C[X]

tensor([[[-0.8149, -1.4727],
         [-0.8149, -1.4727],
         [-0.8149, -1.4727]],

        [[-0.8149, -1.4727],
         [-0.8149, -1.4727],
         [ 0.6220,  1.0828]],

        [[-0.8149, -1.4727],
         [ 0.6220,  1.0828],
         [-1.1231,  0.1798]],

        [[ 0.6220,  1.0828],
         [-1.1231,  0.1798],
         [-1.1231,  0.1798]],

        [[-1.1231,  0.1798],
         [-1.1231,  0.1798],
         [-1.6462, -1.4466]]])

In [29]:
# There are 100 neurons, each accepts a 6-D input sample

W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [30]:
# emb.view(-1, 6): == there are 228146 samples, each sample has 6 dimensions.
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [31]:
h

tensor([[-1.0000,  0.9985, -0.9994,  0.9996, -0.9725,  0.0023,  1.0000, -0.9915,
         -0.9717, -1.0000,  0.9971,  1.0000,  0.6101, -0.5457,  0.1668,  0.9691,
         -0.8079,  0.9292,  1.0000, -1.0000, -0.5810,  0.9999,  1.0000,  0.2651,
         -0.8415, -0.3284,  0.9265, -0.9991, -0.9612,  0.9733, -1.0000, -0.7332,
         -0.9998, -0.9999, -0.8929, -0.0817,  0.9984, -0.5807, -0.9764, -0.9015,
         -0.1723,  0.9993, -0.7332, -1.0000, -0.9901,  0.9899,  0.2998, -0.9971,
          0.1915, -0.9358,  1.0000, -0.9923, -0.9835,  0.4891,  0.9652,  0.9944,
          1.0000,  1.0000,  0.1442, -0.5955, -0.9989, -0.9839,  0.9720,  0.7074,
         -0.2623, -0.6433, -0.9990, -0.1673,  0.8817, -0.9999,  0.3646, -0.9703,
          0.9993, -0.3617,  0.8942, -0.9415,  0.8816, -0.9973,  0.9979, -0.9959,
          0.3781,  0.9011,  1.0000,  0.9994,  0.9962,  0.6486,  0.9954, -1.0000,
         -1.0000, -1.0000, -0.9944,  0.9993, -0.8148, -0.5515, -0.9925,  0.7764,
          0.6764,  0.8880,  

In [32]:
h.shape

torch.Size([5, 100])

In [34]:
# There are 27 neurons, each accepts a 100-D input sample
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
logits = h @ W2 + b2

In [35]:
logits = h @ W2 + b2

In [37]:
# (5,27)
logits.shape

torch.Size([5, 27])

In [38]:
counts = logits.exp()

In [39]:
# (5,27)
# Output the predictions for five samples.
# Each prediction is a 27-D row vector, each element of the vector represents the prob of the next char represented by the element.
# For instance, if prob[0, 1] == 0.1, it means that for the first sample (a character, in our situation it's 'e'), the probability of the next char being 'a' (since it's index is 1) is 0.1.
prob = counts / counts.sum(1, keepdims=True)

In [48]:
prob

tensor([[8.1966e-12, 3.2003e-13, 5.8466e-11, 7.3979e-07, 1.5233e-14, 6.1835e-15,
         2.5206e-12, 8.0280e-09, 7.7497e-13, 5.3522e-14, 5.9385e-08, 9.9926e-01,
         9.1213e-10, 2.3907e-11, 4.2429e-16, 3.3384e-11, 2.2160e-13, 6.3803e-08,
         1.0248e-09, 1.9334e-10, 1.7824e-07, 4.7604e-17, 9.2861e-20, 2.0522e-10,
         2.2402e-12, 1.2692e-11, 7.4164e-04],
        [1.9490e-11, 1.6314e-09, 4.7649e-07, 2.7038e-10, 2.0440e-13, 8.7033e-12,
         6.1506e-12, 1.1086e-09, 2.8329e-17, 1.3611e-10, 9.1783e-09, 7.5510e-08,
         1.4883e-18, 1.6413e-07, 1.6145e-08, 6.9429e-15, 9.1780e-17, 2.5108e-12,
         1.3492e-13, 7.6261e-09, 1.1037e-06, 1.4717e-17, 3.5639e-17, 1.1905e-08,
         1.4942e-05, 1.0704e-05, 9.9997e-01],
        [2.3013e-07, 2.3073e-05, 4.3382e-06, 1.2428e-05, 1.9314e-10, 1.0398e-11,
         9.3780e-05, 1.8331e-03, 5.4539e-07, 1.3202e-08, 7.2372e-04, 1.8942e-05,
         1.4739e-10, 1.5229e-09, 5.0274e-06, 4.4397e-06, 1.4895e-14, 8.8519e-03,
         1.7718e-

In [41]:
prob.shape

torch.Size([5, 27])

In [51]:
# prob[torch.arange(5), Y] == prob[[0,1,2,3,4], Y]. Y is the ground truth, representing the label of the next token.
# Suppose Y == [ 5, 13, 13,  1,  0]. Then:
# 1. prob[0,5] means the prob of the next token of the first sample being 5 (means 'e'). Or it's the accuracy of the model for the first sample.
# 2. prob[1,13] means the prob of the next token of the first sample being 13 (means 'm'). Or it's the accuracy of the model for the 2nd sample.
# 3. ...
loss = -prob[torch.arange(5), Y].log().mean()
loss

tensor(26.1534)

In [49]:
torch.arange(5)

tensor([0, 1, 2, 3, 4])

In [58]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])# training set
Xdev, Ydev = build_dataset(words[n1:n2]) # validation set
Xte, Yte = build_dataset(words[n2:])# test set

torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


In [59]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)

# Here we define a MLP with two layers. The MLP takes a 30-D input and produces a 27-D output.
# A layer of 200 neurons
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)

# A layer of 27 neurons.
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [60]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [61]:
for p in parameters:
  p.requires_grad = True

In [62]:
lri = []
lossi = []
stepi = []

In [84]:
for i in range(5):
  
  # minibatch construct
  
  # torch.randint: returns a tensor filled with random integers generated uniformly between the specified lower and upper bounds.
  # 0: This is the lower bound (inclusive) for the range of random integers. It indicates that the random integers generated will be greater than or equal to 0.
  # Xtr.shape[0]: This represents the upper bound (exclusive) of the range of random integers. 
  # Xtr.shape[0] is the total number of samples in the training dataset (Xtr).
  # (32,): This is a tuple specifying the size of the output tensor. In this case, the output tensor will contain 32 random integers. The comma is necessary to indicate that this is a single-element tuple
  # ix: [1,32]. 
  ix = torch.randint(0, Xtr.shape[0], (32,))
  # print(ix) # ix is the index of a sample or a set of indexes in a batch of sample (batch size=32)
  # forward pass
  input = Xtr[ix] # a sample or a batch of sample. Each sample is a context (len=3)
  # print(Xtr[ix].shape) 
  emb = C[Xtr[ix]] # (32, 3, 10) # create 10-D embeddings for each sample
  # print(emb.shape)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  print(loss.item())
  # 
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

#print(loss.item())

2.018911123275757
2.3627350330352783
2.459622621536255
2.814265251159668
2.1975536346435547


In [91]:
for i in range(5):
  batch_size = 32
  training_set_size = Xtr.shape[0]
  ix = torch.randint(0, training_set_size, (batch_size,))
  
  input = Xtr[ix]
  true_output =  Ytr[ix]
  emb = C[input]
  h = torch.tanh(emb.view(-1,30) @ W1 + b1) # (batch_size * 200)
  h2 = h @ W2 + b2 # (batch_size * 27)
  logits = h2
  
  loss = F.cross_entropy(logits, true_output)
  print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

2.5878899097442627
2.2648394107818604
2.28810715675354
2.794795513153076
2.243070363998413
